This script is to understand the word embedding and how it works.

In [2]:
from mpmath.calculus.extrapolation import nprod
from sentence_transformers import SentenceTransformer, util
import torch
import torch.nn.functional as F

In [3]:

model = SentenceTransformer('sentence-transformers/all-MiniLM-L12-v2')


In [4]:
from datasets import load_dataset

org_path = "D:\\LLM\\thesisPractical\\datasets_for_fine_tuning\\structural_removal_non_contiguous\\processed_4000"

test_dataset_url = org_path + "\\test.jsonl"

data_files = {
    'test': test_dataset_url
}

dataset = load_dataset('json', data_files=data_files)
test_dataset = dataset['test']

In [5]:
true_node1 = "{'viewpoint': None, 'visibility': 'PUBLIC_LITERAL', 'qualifiedName': 'model', 'name': 'model', 'id': 0, 'URI': None, 'eClass': 'Model'}"

In [6]:
predicted1 = "{'viewpoint': None, 'visibility': 'PUBLIC_LITERAL', 'qualifiedName': 'model', 'name': 'model', 'id': 0, 'URI': 'game', 'eClass': 'Class'}"

In [7]:
core_true = "'qualifiedName': 'model', 'name': 'model', 'eClass': 'Model'"

true_node ="'viewpoint': None, 'visibility': 'PUBLIC_LITERAL', 'URI': None"

In [8]:
core_pred = "'qualifiedName': 'house', 'name': 'house', 'eClass': 'Class'"

predicted ="'viewpoint': None, 'visibility': 'PUBLIC_LITERAL', 'URI': 'game'"

TypeError: can't multiply sequence by non-int of type 'float'

In [217]:
sentence = [true_node1,
            predicted1]

embeddings = model.encode(sentence)
print(embeddings)

[[ 1.99090969e-02  2.07003877e-02 -2.71928962e-02  3.34893614e-02
   5.98727865e-03  1.92389544e-02  4.29515960e-03  1.00038750e-02
   1.83490361e-03  5.62479980e-02 -1.20820785e-02 -1.22029833e-01
  -1.18128275e-02  2.38850117e-02  1.24997245e-02  6.01265207e-02
   6.46429369e-03  8.00829530e-02  8.33330676e-02  6.58178553e-02
  -4.15743329e-03  1.12418287e-01 -1.47335473e-02  5.08480817e-02
  -1.40212411e-02  5.36390441e-03  7.46013522e-02  7.11294860e-02
   4.55842428e-02 -5.45359496e-03  9.01842639e-02 -5.95283173e-02
   1.13198757e-02  6.91303536e-02  1.90124623e-02  1.06004085e-02
  -2.57912651e-03 -1.27963066e-01 -1.01103843e-03  1.67769939e-03
   9.08344705e-03  2.79337466e-02 -7.52237141e-02 -2.66184993e-02
   4.22770083e-02 -1.50211742e-02 -9.00433026e-03 -2.26165112e-02
   6.09496497e-02 -3.39921713e-02 -5.57616651e-02 -1.12259351e-01
  -8.17681774e-02 -3.74463461e-02 -2.32335902e-03 -5.07231541e-02
  -2.47772112e-02 -2.73297522e-02 -3.60619873e-02 -3.36168259e-02
   2.53834

In [218]:
import numpy as np

similarity = np.dot(embeddings[0], embeddings[1])/(np.linalg.norm(embeddings[0])*np.linalg.norm(embeddings[1]))

In [219]:
similarity

0.9643105

In [220]:
# other way to get embeddings
sentence = [core_true, true_node, core_pred, predicted]
embeddings = model.encode(sentence, convert_to_tensor=True)

In [221]:
alpha = 0.7
beta = 0.3

final_embedding1 = alpha * embeddings[0] + beta * embeddings[1]
final_embedding2 = alpha * embeddings[2] + beta * embeddings[3]

In [222]:
similarity = util.cos_sim(final_embedding1,final_embedding2).item()

In [223]:
similarity

0.8390852212905884

In [106]:
checkpoint = "D:\\LLM\\thesisPractical\\fine_tuned_models\\Qwen2.5-3B-Instruct-software-model_completion"

In [264]:
def compute_node_similarity_all_calculated(node1, node2, secondary_attrs):
    # Extract key attributes
    eclass_val1 = node1.get("eClass", "")
    eclass_val2 = node2.get("eClass", "")
    
    name_val1 = node1.get("name")
    name_val2 = node2.get("name")
    
    qn_val1 = node1.get("qualifiedName")
    qn_val2 = node2.get("qualifiedName")

    # Determine presence
    name_present = (name_val1 is not None) and (name_val2 is not None)
    qn_present = (qn_val1 is not None) and (qn_val2 is not None)

    # Base weights
    w_eclass = 0.35
    w_name = 0.3
    w_qualifiedName = 0.25
    w_rest = 0.15
        # Adjust weights based on missing attributes
    if not name_present and not qn_present:
        # Both missing: redistribute 0.5 from name & QN
        w_eclass += 0.35
        w_rest += 0.15
        w_name = 0
        w_qualifiedName = 0
    elif not name_present:
        # Name missing: redistribute 0.3
        w_eclass += 0.2
        w_rest += 0.1
        w_name = 0
    elif not qn_present:
        # QN missing: redistribute 0.2
        w_eclass += 0.15
        w_rest += 0.05
        w_qualifiedName = 0
        
        # Now normalize weights so total = 1
    # (Some weights might be zero if attributes are missing)
    total = w_eclass + w_name + w_qualifiedName + w_rest
    w_eclass /= total
    if w_name > 0:
        w_name /= total
    if w_qualifiedName > 0:
        w_qualifiedName /= total
    w_rest /= total
    
    print(w_eclass, w_name, w_qualifiedName, w_rest)
    
      # Compute similarities for eClass, name, qn if present
    def cos_sim_value(val1, val2):
        if val1 is None or val2 is None:
            return 0.0
        if val1 == "" or val2 == "":
            return 0.0
        emb1 = model.encode(val1, convert_to_tensor=True)
        emb2 = model.encode(val2, convert_to_tensor=True)
        return util.cos_sim(emb1, emb2).item()

    eclass_sim = cos_sim_value(eclass_val1, eclass_val2)
    name_sim = cos_sim_value(name_val1, name_val2) if name_present else 0.0
    qn_sim = cos_sim_value(qn_val1, qn_val2) if qn_present else 0.0
    
        # For rest attributes, compute per-attribute similarity
    # We'll only consider attributes from the secondary_attrs list
    # that are present in both nodes.
    rest_attrs_in_both = [attr for attr in secondary_attrs if attr in node1 and attr in node2]
    # If no rest attributes, w_rest is effectively unused.
    # If there are rest attributes, each gets an equal share of w_rest
    per_rest_weight = w_rest / len(rest_attrs_in_both) if rest_attrs_in_both else 0.0

    rest_sim_sum = 0.0
    for attr in rest_attrs_in_both:
        val1 = node1.get(attr)
        val2 = node2.get(attr)
        attr_sim = cos_sim_value(str(val1), str(val2))
        rest_sim_sum += (per_rest_weight * attr_sim)

    # Combine all similarities
    final_similarity = (w_eclass * eclass_sim) + (w_name * name_sim) + (w_qualifiedName * qn_sim) + rest_sim_sum
    return final_similarity


In [268]:
# Example usage:
node1 = {
    "eClass": "Class",
    "name": "Customer",
    "qualifiedName": "model::Customer",
    "visibility": "PUBLIC_LITERAL",
    "isAbstract": False,
    "isActive": False, 
    'isLeaf': False, 
    'isFinalSpecialization': False,
    "message": "Hello"
}

node2 = {
    "eClass": "Class",
    "name": "House",
    "qualifiedName": "model::Client",
    "visibility": "PUBLIC_LITERAL",
    "isAbstract": False,
    "isActive": False, 
    'isLeaf': False, 
    'isFinalSpecialization': False,
    "message": "Hello"
}

secondary_attrs = ["visibility", "isAbstract"]
final_similarity = compute_node_similarity_all_calculated(node1, node2, secondary_attrs)
print("Final Similarity:", final_similarity)

threshold = 0.8
if final_similarity >= threshold:
    print("Nodes are considered the same")
else:
    print("Nodes are considered different")

0.33333333333333337 0.28571428571428575 0.23809523809523814 0.14285714285714288
Final Similarity: 0.7526435568219141
Nodes are considered different


In [261]:
from sentence_transformers import SentenceTransformer, util

model = SentenceTransformer('all-MiniLM-L6-v2')

def compute_node_similarity_rest_at_once(node1, node2, secondary_attrs):
    # Extract key attributes
    eclass_val1 = node1.get("eClass", "")
    eclass_val2 = node2.get("eClass", "")
    
    name_val1 = node1.get("name")
    name_val2 = node2.get("name")
    
    qn_val1 = node1.get("qualifiedName")
    qn_val2 = node2.get("qualifiedName")

    # Combine "rest" attributes into a single string for each node
    # Exclude eClass, name, qualifiedName from the rest
    excluded_keys = {"eClass", "name", "qualifiedName"}
    def rest_text(n):
        return " ".join(f"{k}:{v}" for k,v in n.items() if k not in excluded_keys and k in secondary_attrs)
    
    rest_val1 = rest_text(node1)
    rest_val2 = rest_text(node2)

    # Base weights
    w_eclass = 0.4
    w_name = 0.3
    w_qualifiedName = 0.2
    w_rest = 0.1

    # Adjust weights based on missing attributes
    # If name is missing in either node, we consider that attribute absent
    name_present = (name_val1 is not None) and (name_val2 is not None)
    qn_present = (qn_val1 is not None) and (qn_val2 is not None)

    if not name_present and not qn_present:
        # Both missing: redistribute 0.5 (0.3 + 0.2) from name & QN
        # More weight to eClass and some to rest
        w_eclass += 0.35
        w_rest += 0.15
    elif not name_present:
        # Name missing: redistribute 0.3
        w_eclass += 0.2
        w_rest += 0.1
    elif not qn_present:
        # QN missing: redistribute 0.2
        w_eclass += 0.15
        w_rest += 0.05

    # Normalize weights so they sum to 1 (optional but neat)
    weight_sum = w_eclass + (w_name if name_present else 0) + (w_qualifiedName if qn_present else 0) + w_rest
    w_eclass /= weight_sum
    if name_present:
        w_name /= weight_sum
    if qn_present:
        w_qualifiedName /= weight_sum
    w_rest /= weight_sum

    # Compute embeddings and similarities
    # eClass should always be present or at least empty string
    eclass_sim = util.cos_sim(model.encode(eclass_val1), model.encode(eclass_val2)).item()

    if name_present:
        name_sim = util.cos_sim(model.encode(name_val1), model.encode(name_val2)).item()
    else:
        name_sim = 0  # If absent, no similarity from here

    if qn_present:
        qn_sim = util.cos_sim(model.encode(qn_val1), model.encode(qn_val2)).item()
    else:
        qn_sim = 0

    # Rest similarity (if no secondary attrs, this might be empty)
    if rest_val1.strip() and rest_val2.strip():
        rest_sim = util.cos_sim(model.encode(rest_val1), model.encode(rest_val2)).item()
    else:
        rest_sim = 0  # If no rest attributes present, no contribution

    # Combine similarities using adjusted weights
    final_similarity = (w_eclass * eclass_sim) \
                       + (w_name * name_sim if name_present else 0) \
                       + (w_qualifiedName * qn_sim if qn_present else 0) \
                       + (w_rest * rest_sim)

    return final_similarity

In [262]:
# Example usage:
node1 = {
    "eClass": "Class",
    "name": "Customer",
    "qualifiedName": "model::Customer",
    "visibility": "PUBLIC_LITERAL",
    "isAbstract": False
}

node2 = {
    "eClass": "Class",
    "name": "Client",
    "qualifiedName": "model::Client",
    "visibility": "PUBLIC_LITERAL",
    "isAbstract": False
}

secondary_attrs = ["visibility", "isAbstract"]
final_similarity = compute_node_similarity_rest_at_once(node1, node2, secondary_attrs)
print("Final Similarity:", final_similarity)

threshold = 0.8
if final_similarity >= threshold:
    print("Nodes are considered the same")
else:
    print("Nodes are considered different")

Final Similarity: 0.8312053322792055
Nodes are considered the same


In [37]:
model = SentenceTransformer('paraphrase-multilingual-MiniLM-L12-v2')

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

C:\Users\Admin\anaconda3\envs\LLM\Lib\site-packages\huggingface_hub\file_download.py:157: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Admin\.cache\huggingface\hub\models--sentence-transformers--paraphrase-multilingual-MiniLM-L12-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/4.12k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

model.safetensors:   0%|          | 0.00/471M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/480 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.08M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [124]:
sentence1 = "dog"
sentence2 = "cat"

embeddings = model.encode([sentence1, sentence2])
similarity_customer_client = util.cos_sim(embeddings[0], embeddings[1]).item()

similarity_customer_client

0.30328798294067383

In [115]:
# Define the word pairs
word1 = "The eClass is House, the name is King, the qualifiedName is model::King"
word2 = "The eClass is Mansion, the name is Queen, the qualifiedName is model::Queen"
word3 = "The eClass is Client, the name is queen, the qualifiedName is model::queen"

# Encode the words
embedding_customer = model.encode(word1, convert_to_tensor=True)
embedding_client = model.encode(word2, convert_to_tensor=True)
embedding_clients = model.encode(word3, convert_to_tensor=True)

# Compute cosine similarities
similarity_customer_client = util.cos_sim(embedding_customer, embedding_client).item()
similarity_customer_clients = util.cos_sim(embedding_customer, embedding_clients).item()

print(f"Similarity between '{word1}' and '{word2}': {similarity_customer_client:.4f}")
print(f"Similarity between '{word1}' and '{word3}': {similarity_customer_clients:.4f}")

Similarity between 'The eClass is House, the name is King, the qualifiedName is model::King' and 'The eClass is Mansion, the name is Queen, the qualifiedName is model::Queen': 0.8585
Similarity between 'The eClass is House, the name is King, the qualifiedName is model::King' and 'The eClass is Client, the name is queen, the qualifiedName is model::queen': 0.7436


In [114]:
word1 = "The eClass is House, the name is King, the qualifiedName is model::King"
word2 = "The eClass is Mansion, the name is Queen, the qualifiedName is model::Queen"
word3 = "The eClass is Class, the name is Clients, the qualifiedName is model::Clients"
# Split the variables into substrings
substrings_word1 = word1.split(", ")
substrings_word2 = word2.split(", ")
substrings_word3 = word3.split(", ")

class1 = substrings_word1[0]
class2 = substrings_word2[0]
class3 = substrings_word3[0]
print(class1, class2, class3)

name1 = substrings_word1[1]
name2 = substrings_word2[1]
name3 = substrings_word3[1]

qn1 = substrings_word1[2]
qn2 = substrings_word2[2]
qn3 = substrings_word3[2]

weight_class = 0.4
weight_name = 0.35
weight_qn = 0.25

# Encode the words
embedding_class1 = model.encode(substrings_word1[0], convert_to_tensor=True)
embedding_class2 = model.encode(substrings_word2[0], convert_to_tensor=True)
embedding_class3 = model.encode(substrings_word3[0], convert_to_tensor=True)

# Compute cosine similarities
similarity_customer_client = util.cos_sim(embedding_class1, embedding_class2).item()
similarity_customer_clients = util.cos_sim(embedding_class1, embedding_class3).item()

print(f"Similarity between '{class1}' and '{class2}': {similarity_customer_client:.4f}")
print(f"Similarity between '{class1}' and '{class3}': {similarity_customer_clients:.4f}")

embedding_name1 = model.encode(substrings_word1[1], convert_to_tensor=True)
embedding_name2 = model.encode(substrings_word2[1], convert_to_tensor=True)
embedding_name3 = model.encode(substrings_word3[1], convert_to_tensor=True)

similarity_name1_name2 = util.cos_sim(embedding_name1, embedding_name2).item()
similarity_name1_name3 = util.cos_sim(embedding_name1, embedding_name3).item()

print(f"Similarity between '{name1}' and '{name2}': {similarity_name1_name2:.4f}")
print(f"Similarity between '{name1}' and '{name3}': {similarity_name1_name3:.4f}")

embedding_qn1 = model.encode(substrings_word1[2], convert_to_tensor=True)
embedding_qn2 = model.encode(substrings_word2[2], convert_to_tensor=True)
embedding_qn3 = model.encode(substrings_word3[2], convert_to_tensor=True)

similarity_qn1_qn2 = util.cos_sim(embedding_qn1, embedding_qn2).item()
similarity_qn1_qn3 = util.cos_sim(embedding_qn1, embedding_qn3).item()

print(f"Similarity between '{qn1}' and '{qn2}': {similarity_qn1_qn2:.4f}")
print(f"Similarity between '{qn1}' and '{qn3}': {similarity_qn1_qn3:.4f}")

similarity_with_weights = (weight_class * similarity_customer_client) + (weight_name * similarity_name1_name2) + (weight_qn * similarity_qn1_qn2)
print(f"Similarity with weights1 and 2: {similarity_with_weights:.4f}")


The eClass is House The eClass is Mansion The eClass is Class
Similarity between 'The eClass is House' and 'The eClass is Mansion': 0.7725
Similarity between 'The eClass is House' and 'The eClass is Class': 0.5498
Similarity between 'the name is King' and 'the name is Queen': 0.7253
Similarity between 'the name is King' and 'the name is Clients': 0.1733
Similarity between 'the qualifiedName is model::King' and 'the qualifiedName is model::Queen': 0.8707
Similarity between 'the qualifiedName is model::King' and 'the qualifiedName is model::Clients': 0.7784
Similarity with weights1 and 2: 0.7805


## OpenAI - Embeddings

In [85]:
from numpy import dot
from numpy.linalg import norm
from openai import OpenAI
client = OpenAI(api_key="")

def get_embedding(word):
    response = client.embeddings.create(
        input=word,
        model="text-embedding-ada-002"
    )
    return response.data[0].embedding



In [122]:
# Example words
word1 = "Dog"
word2 = "cat"

embedding1 = get_embedding(word1)
embedding2 = get_embedding(word2)

In [125]:
def cosine_similarity(vec1, vec2):
    return dot(vec1, vec2) / (norm(vec1) * norm(vec2))

similarity = cosine_similarity(embedding1, embedding2)

similarity_qn1_qn3 = util.cos_sim(embedding1, embedding2).item()

print(f"Similarity between '{word1}' and '{word2}':", similarity)

Similarity between 'Dog' and 'cat': 0.8651525539979711


BaseModelOutputWithPast(last_hidden_state=tensor([[[ 2.2305, -0.8589,  2.2832,  ..., -1.3838, -0.5322, -0.3289]]],
       dtype=torch.float16), past_key_values=((tensor([[[[-3.9141e+00, -1.0303e+00,  1.7227e+00,  2.4941e+00, -3.6055e+00,
           -1.0049e+00,  2.9663e-01,  2.4512e+00, -9.5508e-01, -9.4824e-01,
           -4.1328e+00,  7.0703e-01,  2.6250e+00, -2.6489e-01, -9.0820e-01,
           -1.5273e+00,  4.0430e-01,  0.0000e+00, -4.7461e-01,  1.1973e+00,
            2.8828e+00, -2.8122e-02, -4.4604e-01, -5.5542e-03, -1.6052e-01,
           -8.0566e-02,  6.6309e-01, -2.8203e+00, -7.1045e-02, -2.3730e-01,
            5.7764e-01, -6.7871e-01,  1.4417e-01, -7.8418e-01, -4.5898e-02,
            5.8545e-01, -4.1553e-01, -5.0195e-01, -4.4336e+00,  9.9316e-01,
           -1.7432e-01,  1.2622e-01,  1.3535e+00, -7.4902e-01,  9.9951e-01,
           -1.6260e-01, -8.5547e-01, -6.7480e-01,  1.1660e+00, -1.7266e+01,
           -4.5156e+00, -1.4629e+00,  6.9062e+00, -7.5273e+00,  3.6582e+00,
  

AttributeError: 'BaseModelOutputWithPast' object has no attribute 'logits'

tensor([[[ 2.2305, -0.8589,  2.2832,  ..., -1.3838, -0.5322, -0.3289]]],
       device='cuda:0', dtype=torch.float16)
tensor([[[ 2.2305, -0.8589,  2.2832,  ..., -1.3838, -0.5322, -0.3289]]],
       device='cuda:0', dtype=torch.float16)


Cosine similarity: 1.0
